# Scraper Hacker News

In [1]:
# Dépendances
import requests
from bs4 import BeautifulSoup

## Liste des choses à récupérer:

* Url
* Title
* Author
* Comments
* Upvotes
* Id

## 1. Récupérer le HTML de la page

In [3]:
r = requests.get('https://news.ycombinator.com/')

## 2. Utiliser BeautifulSoup pour parser le HTML

In [4]:
soup = BeautifulSoup(r.text)

## 3. Extraire les informations

In [42]:
data = []

def get_comments(elem):
    """Retrieving number of comments from target sibling element."""
    txt = elem.get_text().strip()
    
    if txt == 'discuss':
        return 0

    return int(txt.split('comment')[0].strip())

def get_user(elem):
    """Retrieving user id from target sibling element."""
    if elem is None:
        return ''
    
    return elem.get_text().strip()

for tr in soup.select('tr.athing'):
    sibling = tr.find_next_sibling()
    
    link = tr.select_one('a.storylink')
    
    post = {
        'id': tr.get('id'),
        'url': link.get('href'),
        'title': link.get_text(),
        'points': int(sibling.select_one('.score').get_text().split(' points')[0].strip()),
        'comments': get_comments(sibling.select('a')[-1]),
        'author': get_user(sibling.select_one('a[href^="user?"]'))
    }
    
    data.append(post)

## 4. Dump en json

In [43]:
import json

with open('./hackernews.json', 'w') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)